INstallation du client PROCGEN

In [1]:
!pip install procgen

     ---------------------------------------- 0.0/30.0 MB ? eta -:--:--
     --------------------------------------- 0.0/30.0 MB 660.6 kB/s eta 0:00:46
     ---------------------------------------- 0.2/30.0 MB 2.1 MB/s eta 0:00:15
     ---------------------------------------- 0.3/30.0 MB 2.1 MB/s eta 0:00:14
      --------------------------------------- 0.4/30.0 MB 2.4 MB/s eta 0:00:13
      --------------------------------------- 0.7/30.0 MB 2.9 MB/s eta 0:00:10
     - -------------------------------------- 0.9/30.0 MB 3.4 MB/s eta 0:00:09
     - -------------------------------------- 1.0/30.0 MB 3.6 MB/s eta 0:00:09
     - -------------------------------------- 1.2/30.0 MB 3.5 MB/s eta 0:00:09
     - -------------------------------------- 1.5/30.0 MB 3.7 MB/s eta 0:00:08
     -- ------------------------------------- 1.9/30.0 MB 4.1 MB/s eta 0:00:07
     -- ------------------------------------- 2.2/30.0 MB 4.4 MB/s eta 0:00:07
     --- ------------------------------------ 2.6/30.0 MB 

Instanciation of the environment

In [2]:
import torch
from procgen import ProcgenEnv
from procgen_wrappers import VecExtractDictObs, TransposeFrame, ScaledFloatFrame

env = ProcgenEnv(
        num_envs=1,
        env_name="fruitbot",
        start_level=0,
        num_levels=100,
        distribution_mode='easy',
    )

env = VecExtractDictObs(env, "rgb")
env = TransposeFrame(env)
env = ScaledFloatFrame(env)

The Agent class is defined in the agent.py file.
An agent is composed of a feature extractor and a policy. The feature extractor is a neural network that takes the image as input and outputs a vector of features. The policy is a neural network that takes the features as input and outputs a score for each action. The action with the highest score is the one that is chosen.

You can use it as follows:

In [4]:
from agent import Agent
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device=torch.device('cpu')
agent = Agent()
agent.load_state_dict(torch.load('agent_weights.pth', map_location=device))
agent.eval()

Agent(
  (features): ImpalaModel(
    (block1): ImpalaBlock(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (res1): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
      (res2): ResidualBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      )
    )
    (block2): ImpalaBlock(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (res1): ResidualBlock(
        (conv1): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(32, 32, kernel_size=(

The following code shows how to use the agent to play a game

In [7]:
obs = env.reset()

while True:
    obs = torch.FloatTensor(obs).to(device)
    action = agent(obs).argmax(1).cpu().numpy()
    obs, _, done ,_ = env.step(action)
    if done[0]:
        break

env.close()

## Code to record a gif

In [8]:
import imageio
from IPython.display import Image
from tqdm.notebook import tqdm
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np

def obs_to_image(obs):
  return (obs[0].transpose(1,2,0) * 255).astype(np.uint8)

def display_trajectory(frames, fps=25):
  imageio.mimwrite('./tmp.gif',
                [obs_to_image(frame) for i, frame in enumerate(frames)],
                fps=fps)
  return(Image(open('tmp.gif','rb').read(), width=500, height=500))

  frames = []
obs = env.reset()

while True:
    frames.append(obs)
    obs = torch.FloatTensor(obs).to('cuda')
    action = agent(obs).argmax(1).cpu().numpy()
    obs, _, done ,_ = env.step(action)
    img = env.render()
    if done[0]:
        break

env.close()

display_trajectory(frames)

NameError: name 'frames' is not defined

## Objectives
You can use the tools seen in the course, and you can also use other tools if you want. You are expected to produce three types of explanations.
Don't be surprised if some methods don't work well on every frame (like grad-CAM), it is normal.